# 抓取豆瓣电影中王晶的导演作品分析

In [2]:
'''
    获取王晶导演的作品并存储
'''
import re
import os
import csv
import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

# 王晶 Jing Wong 的导演作品 链接
url = 'https://movie.douban.com/celebrity/1274331/movies?sortby=vote&format=pic&role=D'

chrome_opt = Options()
driver = webdriver.Chrome(executable_path='./chromedriver', options=chrome_opt)
driver.get(url)
        
next_page = driver.find_element_by_xpath('//*[@class="next"]')

# 创建 csv
with open('./JingWong.csv', 'w', newline='', encoding='utf-8-sig') as csvfile:
    writer = csv.writer(csvfile, delimiter=',')
    writer.writerow(['电影名称', '主演演员'])

num = 1
while next_page:
    print(f'当前页为 第 {num} 页')
    time.sleep(1)
    movie_lists = driver.find_elements_by_xpath('//*[@id="content"]/div/div/div/ul/li/dl/dd/h6/a')
    actor_lists = driver.find_elements_by_xpath('//*[@id="content"]/div/div/div/ul/li/dl/dd/dl/dd[2]')
    for movie_t, actor_list in zip(movie_lists, actor_lists):
        movie = movie_t.text
        actors = re.sub('[a-zA-z\.\ \-]|/"$', '', actor_list.text).split('/')
        with open('JingWong.csv', 'a+', encoding='utf-8-sig') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow([movie, [x for x in actors if len(x) > 1]])

    next_page = driver.find_element_by_xpath('//*[@class="next"]')
    next_page.click()
    num += 1
    if num > 10:
        break

driver.quit()

当前页为 第 1 页
当前页为 第 2 页
当前页为 第 3 页
当前页为 第 4 页
当前页为 第 5 页
当前页为 第 6 页
当前页为 第 7 页
当前页为 第 8 页
当前页为 第 9 页
当前页为 第 10 页


In [3]:
'''
    利用 Apriori 算法分析
    找出演员任用关联规则
'''
import re
import pandas as pd
from efficient_apriori import apriori

# 加载数据集
data_o = pd.read_csv('./JingWong.csv', encoding='utf-8-sig', converters={'主演演员': lambda x: list(map(str, x.split(',')))})
data_n = data_o['主演演员'].values.tolist()
data = []
for d in data_n:
    names = []
    for name in d:
        names.append(re.sub("['|\[|\]\ ]", '', name))
    data.append(names)
# 挖掘频繁项集和频繁规则
itemsets, rules = apriori(data[:30], min_support=0.1, min_confidence=1)

In [4]:
itemsets

{1: {('吴孟达',): 3,
  ('周星驰',): 9,
  ('张敏',): 7,
  ('刘德华',): 9,
  ('陈百祥',): 4,
  ('邱淑贞',): 8,
  ('叶德娴',): 4,
  ('周海媚',): 3,
  ('梁家辉',): 4,
  ('李连杰',): 4,
  ('张卫健',): 3,
  ('张学友',): 3,
  ('方中信',): 3},
 2: {('吴孟达', '周星驰'): 3, ('周星驰', '张敏'): 3, ('刘德华', '张敏'): 3, ('李连杰', '邱淑贞'): 3}}

In [5]:
rules

[{吴孟达} -> {周星驰}]

最终发现出现吴孟达的王晶电影里，很大程度上会存在演员周星驰。